Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-hierarchical-timeseries/auto-ml-forecasting-hierarchical-timeseries.png)

<font color="red" size="5"><strong>!Important!</strong> </br>This notebook is outdated and is not supported by the AutoML Team. Please use the supported version ([link](https://github.com/Azure/azureml-examples/tree/main/sdk/python/jobs/pipelines/1k_demand_forecasting_with_pipeline_components/automl-forecasting-demand-many-models-in-pipeline)).</font>

# Many Models - Automated ML
**_Generate many models time series forecasts with Automated Machine Learning_**

---

For this notebook we are using a synthetic dataset portraying sales data to predict the the quantity of a vartiety of product skus across several states, stores, and product categories.

**NOTE: There are limits on how many runs we can do in parallel per workspace, and we currently recommend to set the parallelism to maximum of 320 runs per experiment per workspace. If users want to have more parallelism and increase this limit they might encounter Too Many Requests errors (HTTP 429).**

### Prerequisites
You'll need to create a compute Instance by following [these](https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-create-manage-compute-instance?tabs=python) instructions.

## 1.0 Set up workspace, datastore, experiment

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
import pandas as pd

# Set up your workspace
ws = Workspace.from_config()
ws.get_details()

# Set up your datastores
dstore = ws.get_default_datastore()

output = {}
output["SDK version"] = azureml.core.VERSION
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Default datastore name"] = dstore.name
output["SDK Version"] = azureml.core.VERSION
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

### Choose an experiment

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, "automl-many-models")

print("Experiment name: " + experiment.name)

## 2.0 Data

This notebook uses simulated orange juice sales data to walk you through the process of training many models on Azure Machine Learning using Automated ML. 

The time series data used in this example was simulated based on the University of Chicago's Dominick's Finer Foods dataset which featured two years of sales of 3 different orange juice brands for individual stores. The full simulated dataset includes 3,991 stores with 3 orange juice brands each thus allowing 11,973 models to be trained to showcase the power of the many models pattern.

  
In this notebook, two datasets will be created: one with all 11,973 files and one with only 10 files that can be used to quickly test and debug. For each dataset, you'll be walked through the process of:

1. Registering the blob container as a Datastore to the Workspace
2. Registering a tabular dataset to the Workspace

### 2.1 Data Preparation
The OJ data is available in the public blob container. The data is split to be used for training and for inferencing. For the current dataset, the data was split on time column ('WeekStarting') before and after '1992-5-28' .

The container has
<ol>
    <li><b>'oj-data-tabular'</b> and <b>'oj-inference-tabular'</b> folders that contains training and inference data respectively for the 11,973 models. </li>
    <li>It also has <b>'oj-data-small-tabular'</b> and <b>'oj-inference-small-tabular'</b> folders that has training and inference data for 10 models.</li>
</ol>

To create the [TabularDataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabular_dataset.tabulardataset?view=azure-ml-py) needed for the ParallelRunStep, you first need to register the blob container to the workspace.

<b> To use your own data, put your own data in a blobstore folder. As shown it can be one file or multiple files. We can then register datastore using that blob as shown below.
    
<h3> How sample data in blob store looks like</h3>

['oj-data-tabular'](https://ms.portal.azure.com/#blade/Microsoft_Azure_Storage/ContainerMenuBlade/overview/storageAccountId/%2Fsubscriptions%2F102a16c3-37d3-48a8-9237-4c9b1e8e80e0%2FresourceGroups%2FAutoMLSampleNotebooksData%2Fproviders%2FMicrosoft.Storage%2FstorageAccounts%2Fautomlsamplenotebookdata/path/automl-sample-notebook-data/etag/%220x8D84EAA65DE50B7%22/defaultEncryptionScope/%24account-encryption-key/denyEncryptionScopeOverride//defaultId//publicAccessVal/Container)</b>
![image-4.png](mm-1.png)

['oj-inference-tabular'](https://ms.portal.azure.com/#blade/Microsoft_Azure_Storage/ContainerMenuBlade/overview/storageAccountId/%2Fsubscriptions%2F102a16c3-37d3-48a8-9237-4c9b1e8e80e0%2FresourceGroups%2FAutoMLSampleNotebooksData%2Fproviders%2FMicrosoft.Storage%2FstorageAccounts%2Fautomlsamplenotebookdata/path/automl-sample-notebook-data/etag/%220x8D84EAA65DE50B7%22/defaultEncryptionScope/%24account-encryption-key/denyEncryptionScopeOverride//defaultId//publicAccessVal/Container)
![image-3.png](mm-2.png)

['oj-data-small-tabular'](https://ms.portal.azure.com/#blade/Microsoft_Azure_Storage/ContainerMenuBlade/overview/storageAccountId/%2Fsubscriptions%2F102a16c3-37d3-48a8-9237-4c9b1e8e80e0%2FresourceGroups%2FAutoMLSampleNotebooksData%2Fproviders%2FMicrosoft.Storage%2FstorageAccounts%2Fautomlsamplenotebookdata/path/automl-sample-notebook-data/etag/%220x8D84EAA65DE50B7%22/defaultEncryptionScope/%24account-encryption-key/denyEncryptionScopeOverride//defaultId//publicAccessVal/Container)

![image-5.png](mm-3.png)

['oj-inference-small-tabular'](https://ms.portal.azure.com/#blade/Microsoft_Azure_Storage/ContainerMenuBlade/overview/storageAccountId/%2Fsubscriptions%2F102a16c3-37d3-48a8-9237-4c9b1e8e80e0%2FresourceGroups%2FAutoMLSampleNotebooksData%2Fproviders%2FMicrosoft.Storage%2FstorageAccounts%2Fautomlsamplenotebookdata/path/automl-sample-notebook-data/etag/%220x8D84EAA65DE50B7%22/defaultEncryptionScope/%24account-encryption-key/denyEncryptionScopeOverride//defaultId//publicAccessVal/Container)
![image-6.png](mm-4.png)

#### 2.2 Register the blob container as DataStore

A Datastore is a place where data can be stored that is then made accessible to a compute either by means of mounting or copying the data to the compute target.

Please refer to [Datastore](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore(class)?view=azure-ml-py) documentation on how to access data from Datastore.

In this next step, we will be registering blob storage as datastore to the Workspace.

In [ ]:
from azureml.core import Datastore

# Please change the following to point to your own blob container and pass in account_key
blob_datastore_name = "automl_many_models"
container_name = "automl-sample-notebook-data"
account_name = "automlsamplenotebookdata"

oj_datastore = Datastore.register_azure_blob_container(
    workspace=ws,
    datastore_name=blob_datastore_name,
    container_name=container_name,
    account_name=account_name,
    create_if_not_exists=True,
)

#### 2.3 Using tabular datasets 

Now that the datastore is available from the Workspace, [TabularDataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabular_dataset.tabulardataset?view=azure-ml-py) can be created. Datasets in Azure Machine Learning are references to specific data in a Datastore.  We are using TabularDataset, so that users who have their data which can be in one or many files (*.parquet or *.csv) and have not split up data according to group columns needed for training, can do so using out of box support for 'partiion_by' feature of TabularDataset shown in section 5.0 below.

In [ ]:
from azureml.core import Dataset

ds_name_small = "oj-data-small-tabular"
input_ds_small = Dataset.Tabular.from_delimited_files(
    path=oj_datastore.path(ds_name_small + "/"), validate=False
)

inference_name_small = "oj-inference-small-tabular"
inference_ds_small = Dataset.Tabular.from_delimited_files(
    path=oj_datastore.path(inference_name_small + "/"), validate=False
)

#### 2.4 Configure data with ``OutputFileDatasetConfig`` objects
This step shows how to configure output data from a pipeline step. One of the use cases for this step is when you want to do some preprocessing before feeding the data to training step. Intermediate data (or output of a step) is represented by an ``OutputFileDatasetConfig`` object. ``output_data`` is produced as the output of a step. Optionally, this data can be registered as a dataset by calling the ``register_on_complete`` method. If you create an ``OutputFileDatasetConfig`` in one step and use it as an input to another step, that data dependency between steps creates an implicit execution order in the pipeline.

``OutputFileDatasetConfig`` objects return a directory, and by default write output to the default datastore of the workspace.

Since instance creation for class ``OutputTabularDatasetConfig`` is not allowed, we first create an instance of this class. Then we use the ``read_parquet_files`` method to read the parquet file into ``OutputTabularDatasetConfig``.

In [ ]:
from azureml.data.output_dataset_config import OutputFileDatasetConfig

output_data = OutputFileDatasetConfig(
    name="processed_data", destination=(dstore, "outputdataset/{run-id}/{output-name}")
).as_upload()
# output_data_dataset = output_data.register_on_complete(
#     name='processed_data', description = 'files from prev step')
output_data = output_data.read_parquet_files()

## 3.0 Build the training pipeline
Now that the dataset, WorkSpace, and datastore are set up, we can put together a pipeline for training.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

### Choose a compute target

You will need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets#amlcompute) for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.

\*\*Creation of AmlCompute takes approximately 5 minutes.**

If the AmlCompute with that name is already in your workspace this code will skip the creation process. As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this [article](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Name your cluster
compute_name = "mm-compute-v1"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("Found compute target: " + compute_name)
else:
    print("Creating a new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D14_V2", max_nodes=20
    )
    # Create the compute target
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20
    )

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

### Configure the training run's environment
The next step is making sure that the remote training run has all the dependencies needed by the training steps. Dependencies and the runtime context are set by creating and configuring a RunConfiguration object.

The code below shows two options for handling dependencies. As presented, with ``USE_CURATED_ENV = True``, the configuration is based on a [curated environment](https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments). Curated environments have prebuilt Docker images in the [Microsoft Container Registry](https://hub.docker.com/publishers/microsoftowner). For more information, see [Azure Machine Learning curated environments](https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments).

The path taken if you change ``USE_CURATED_ENV`` to False shows the pattern for explicitly setting your dependencies. In that scenario, a new custom Docker image will be created and registered in an Azure Container Registry within your resource group (see [Introduction to private Docker container registries in Azure](https://docs.microsoft.com/en-us/azure/container-registry/container-registry-intro)). Building and registering this image can take quite a few minutes.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment

aml_run_config = RunConfiguration()
aml_run_config.target = compute_target

USE_CURATED_ENV = True
if USE_CURATED_ENV:
    curated_environment = Environment.get(
        workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu"
    )
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False

    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=["pandas", "scikit-learn"],
        pip_packages=["azureml-sdk", "azureml-dataset-runtime[fuse,pandas]"],
        pin_sdk_version=False,
    )

### Set up training parameters

We need to provide ``ForecastingParameters``, ``AutoMLConfig`` and ``ManyModelsTrainParameters`` objects. For the forecasting task we also need to define several settings including the name of the time column, the maximum forecast horizon, and the partition column name(s) definition.

#### ``ForecastingParameters`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **forecast_horizon**               | The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly). Periods are inferred from your data. |
| **time_column_name**               | The name of your time column. |
| **time_series_id_column_names**    | The column names used to uniquely identify timeseries in data that has multiple rows with the same timestamp. |
| **cv_step_size**                   | Number of periods between two consecutive cross-validation folds. The default value is \"auto\", in which case AutoMl determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value. |

#### ``AutoMLConfig`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **task**                           | forecasting |
| **primary_metric**                 | This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i> |
| **blocked_models**                 | Blocked models won't be used by AutoML. |
| **iteration_timeout_minutes**      | Maximum amount of time in minutes that the model can train. This is optional but provides customers with greater control on exit criteria. |
| **iterations**                     | Number of models to train. This is optional but provides customers with greater control on exit criteria. |
| **experiment_timeout_hours**       | Maximum amount of time in hours that each experiment can take before it terminates. This is optional but provides customers with greater control on exit criteria. **It does not control the overall timeout for the pipeline run, instead controls the timeout for each training run per partitioned time series.** |
| **label_column_name**              | The name of the label column. |
| **n_cross_validations**            | Number of cross validation splits. The default value is \"auto\", in which case AutoMl determines the number of cross-validations automatically, if a validation set is not provided. Or users could specify an integer value. Rolling Origin Validation is used to split time-series in a temporally consistent way. |
| **enable_early_stopping**          | Flag to enable early termination if the primary metric is no longer improving. |
| **enable_engineered_explanations** | Engineered feature explanations will be downloaded if enable_engineered_explanations flag is set to True. By default it is set to False to save storage space. |
| **track_child_runs**               | Flag to disable tracking of child runs. Only best run is tracked if the flag is set to False (this includes the model and metrics of the run). |
| **pipeline_fetch_max_batch_size**  | Determines how many pipelines (training algorithms) to fetch at a time for training, this helps reduce throttling when training at large scale. |


#### ``ManyModelsTrainParameters`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **automl_settings**                | The ``AutoMLConfig`` object defined above. |
| **partition_column_names**         | The names of columns used to group your models. For timeseries, the groups must not split up individual time-series. That is, each group must contain one or more whole time-series. |

In [ ]:
from azureml.train.automl.runtime._many_models.many_models_parameters import (
    ManyModelsTrainParameters,
)
from azureml.automl.core.forecasting_parameters import ForecastingParameters
from azureml.train.automl.automlconfig import AutoMLConfig

partition_column_names = ["Store", "Brand"]

forecasting_parameters = ForecastingParameters(
    time_column_name="WeekStarting",
    forecast_horizon=6,
    time_series_id_column_names=partition_column_names,
    cv_step_size="auto",
)

automl_settings = AutoMLConfig(
    task="forecasting",
    primary_metric="normalized_root_mean_squared_error",
    iteration_timeout_minutes=10,
    iterations=15,
    experiment_timeout_hours=0.25,
    label_column_name="Quantity",
    n_cross_validations="auto",  # Feel free to set to a small integer (>=2) if runtime is an issue.
    track_child_runs=False,
    forecasting_parameters=forecasting_parameters,
)

mm_paramters = ManyModelsTrainParameters(
    automl_settings=automl_settings, partition_column_names=partition_column_names
)

### Construct your pipeline steps
Once you have the compute resource and environment created, you're ready to define your pipeline's steps. There are many built-in steps available via the Azure Machine Learning SDK, as you can see on the [reference documentation for the azureml.pipeline.steps package](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps?view=azure-ml-py). The most flexible class is [PythonScriptStep](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.python_script_step.pythonscriptstep?view=azure-ml-py), which runs a Python script.

Your data preparation code is in a subdirectory (in this example, "data_preprocessing_tabular.py" in the directory "./scripts"). As part of the pipeline creation process, this directory is zipped and uploaded to the compute_target and the step runs the script specified as the value for ``script_name``.

The ``arguments`` values specify the inputs and outputs of the step. In the example below, the baseline data is the ``input_ds_small`` dataset. The script data_preprocessing_tabular.py does whatever data-transformation tasks are appropriate to the task at hand and outputs the data to ``output_data``, of type ``OutputFileDatasetConfig``. For more information, see [Moving data into and between ML pipeline steps (Python)](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-move-data-in-out-of-pipelines). The step will run on the machine defined by ``compute_target``, using the configuration ``aml_run_config``.

Reuse of previous results (``allow_reuse``) is key when using pipelines in a collaborative environment since eliminating unnecessary reruns offers agility. Reuse is the default behavior when the ``script_name``, ``inputs``, and the parameters of a step remain the same. When reuse is allowed, results from the previous run are immediately sent to the next step. If ``allow_reuse`` is set to False, a new run will always be generated for this step during pipeline execution.

> Note that we only support partitioned FileDataset and TabularDataset without partition when using such output as input.

> Note that we **drop column** "Revenue" from the dataset in this step to avoid information leak as "Quantity" = "Revenue" / "Price". **Please modify the logic based on your data**.

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

dataprep_source_dir = "./scripts"
entry_point = "data_preprocessing_tabular.py"
ds_input = input_ds_small.as_named_input("train_10_models")

data_prep_step = PythonScriptStep(
    script_name=entry_point,
    source_directory=dataprep_source_dir,
    arguments=["--input", ds_input, "--output", output_data],
    compute_target=compute_target,
    runconfig=aml_run_config,
    allow_reuse=False,
)

input_ds_small = output_data

### Set up many models pipeline

Parallel run step is leveraged to train multiple models at once. To configure the ParallelRunConfig you will need to determine the appropriate number of workers and nodes for your use case. The ``process_count_per_node`` is based off the number of cores of the compute VM. The node_count will determine the number of master nodes to use, increasing the node count will speed up the training process.

| Property                           | Description|
| :---------------                   | :------------------- |
| **experiment**                     | The experiment used for training. |
| **train_data**                     | The file dataset to be used as input to the training run. |
| **node_count**                     | The number of compute nodes to be used for running the user script. We recommend to start with 3 and increase the node_count if the training time is taking too long. |
| **process_count_per_node**         | Process count per node, we recommend 2:1 ratio for number of cores: number of processes per node. eg. If node has 16 cores then configure 8 or less process count per node for optimal performance. |
| **train_pipeline_parameters**      | The set of configuration parameters defined in the previous section. |
| **run_invocation_timeout**         | Maximum amount of time in seconds that the ``ParallelRunStep`` class is allowed. This is optional but provides customers with greater control on exit criteria. This must be greater than ``experiment_timeout_hours`` by at least 300 seconds. |

Calling this method will create a new aggregated dataset which is generated dynamically on pipeline execution.

**Note**: Total time taken for the **training step** in the pipeline to complete = $ \frac{t}{ p \times n } \times ts $
where,
- $ t $ is time taken for training one partition (can be viewed in the training logs)
- $ p $ is ``process_count_per_node``
- $ n $ is ``node_count``
- $ ts $ is total number of partitions in time series based on ``partition_column_names``

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder


training_pipeline_steps = AutoMLPipelineBuilder.get_many_models_train_steps(
    experiment=experiment,
    train_data=input_ds_small,
    compute_target=compute_target,
    node_count=2,
    process_count_per_node=8,
    run_invocation_timeout=1200,
    train_pipeline_parameters=mm_paramters,
)

In [ ]:
from azureml.pipeline.core import Pipeline

training_pipeline = Pipeline(ws, steps=training_pipeline_steps)

### Submit the pipeline to run
Next we submit our pipeline to run. The whole training pipeline takes about 40m using a STANDARD_D16S_V3 VM with our current ParallelRunConfig setting.

In [ ]:
training_run = experiment.submit(training_pipeline)

In [ ]:
training_run.wait_for_completion(show_output=False)

Check the run status, if training_run is in completed state, continue to forecasting. If training_run is in another state, check the portal for failures.

## 5.0 Publish and schedule the train pipeline (Optional)

### 5.1 Publish the pipeline

Once you have a pipeline you're happy with, you can publish a pipeline so you can call it programmatically later on. See this [tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-your-first-pipeline#publish-a-pipeline) for additional information on publishing and calling pipelines.

In [ ]:
# published_pipeline = training_pipeline.publish(name = 'automl_train_many_models',
#                                                description = 'train many models',
#                                                version = '1',
#                                                continue_on_step_failure = False)

### 5.2 Schedule the pipeline
You can also [schedule the pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-schedule-pipelines) to run on a time-based or change-based schedule. This could be used to automatically retrain models every month or based on another trigger such as data drift.

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence

# training_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="automl_training_recurring_schedule",
#                                      description="Schedule Training Pipeline to run on the first day of every month",
#                                      pipeline_id=training_pipeline_id,
#                                      experiment_name=experiment.name,
#                                      recurrence=recurrence)

## 6.0 Forecasting

### Set up output dataset for inference data
Output of inference can be represented as [OutputFileDatasetConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.output_dataset_config.outputdatasetconfig?view=azure-ml-py) object and OutputFileDatasetConfig can be registered as a dataset. 

In [ ]:
from azureml.data import OutputFileDatasetConfig

output_inference_data_ds = OutputFileDatasetConfig(
    name="many_models_inference_output", destination=(dstore, "oj/inference_data/")
).register_on_complete(name="oj_inference_data_ds")

For many models we need to provide the ManyModelsInferenceParameters object.

#### ``ManyModelsInferenceParameters`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **partition_column_names**         | List of column names that identifies groups. |
| **target_column_name**             | \[Optional] Column name only if the inference dataset has the target. |
| **time_column_name**               | \[Optional] Time column name only if it is timeseries. |
| **inference_type**                 | \[Optional] Which inference method to use on the model. Possible values are 'forecast', 'predict_proba', and 'predict'. |
| **forecast_mode**                  | \[Optional] The type of forecast to be used, either 'rolling' or 'recursive'; defaults to 'recursive'. |
| **step**                           | \[Optional] Number of periods to advance the forecasting window in each iteration **(for rolling forecast only)**; defaults to 1. |

#### ``get_many_models_batch_inference_steps`` arguments
| Property                           | Description|
| :---------------                   | :------------------- |
| **experiment**                     | The experiment used for inference run. |
| **inference_data**                 | The data to use for inferencing. It should be the same schema as used for training.
| **compute_target**                 | The compute target that runs the inference pipeline. |
| **node_count**                     | The number of compute nodes to be used for running the user script. We recommend to start with the number of cores per node (varies by compute sku). |
| **process_count_per_node**         | \[Optional] The number of processes per node. By default it's 2 (should be at most half of the number of cores in a single node of the compute cluster that will be used for the experiment).
| **inference_pipeline_parameters**  | \[Optional] The ``ManyModelsInferenceParameters`` object defined above. |
| **append_row_file_name**           | \[Optional] The name of the output file (optional, default value is 'parallel_run_step.txt'). Supports 'txt' and 'csv' file extension. A 'txt' file extension generates the output in 'txt' format with space as separator without column names. A 'csv' file extension generates the output in 'csv' format with comma as separator and with column names. |
| **train_run_id**                   | \[Optional] The run id of the **training pipeline**. By default it is the latest successful training pipeline run in the experiment. |
| **train_experiment_name**          | \[Optional] The train experiment that contains the train pipeline. This one is only needed when the train pipeline is not in the same experiement as the inference pipeline. |
| **run_invocation_timeout**         | \[Optional] Maximum amount of time in seconds that the ``ParallelRunStep`` class is allowed. This is optional but provides customers with greater control on exit criteria. |
| **output_datastore**               | \[Optional] The ``Datastore`` or ``OutputDatasetConfig`` to be used for output. If specified any pipeline output will be written to that location. If unspecified the default datastore will be used. |
| **arguments**                      | \[Optional] Arguments to be passed to inference script. Possible argument is '--forecast_quantiles' followed by quantile values. |

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder
from azureml.train.automl.runtime._many_models.many_models_parameters import (
    ManyModelsInferenceParameters,
)

mm_parameters = ManyModelsInferenceParameters(
    partition_column_names=["Store", "Brand"],
    time_column_name="WeekStarting",
    target_column_name="Quantity",
)

output_file_name = "parallel_run_step.csv"

inference_steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(
    experiment=experiment,
    inference_data=inference_ds_small,
    node_count=2,
    process_count_per_node=8,
    compute_target=compute_target,
    run_invocation_timeout=300,
    output_datastore=output_inference_data_ds,
    train_run_id=training_run.id,
    train_experiment_name=training_run.experiment.name,
    inference_pipeline_parameters=mm_parameters,
    append_row_file_name=output_file_name,
    arguments=["--forecast_quantiles", 0.1, 0.9],
)

In [ ]:
from azureml.pipeline.core import Pipeline

inference_pipeline = Pipeline(ws, steps=inference_steps)

In [ ]:
inference_run = experiment.submit(inference_pipeline)
inference_run.wait_for_completion(show_output=False)

## Retrieve results

The forecasting pipeline forecasts the orange juice quantity for a Store by Brand. The pipeline returns one file with the predictions for each store and outputs the result to the forecasting_output Blob container. The details of the blob container is listed in 'forecasting_output.txt' under Outputs+logs. 

The following code snippet:
1. Downloads the contents of the output folder that is passed in the parallel run step 
2. Reads the output file that has the predictions as pandas dataframe and 
3. Displays the top 10 rows of the predictions

In [ ]:
from azureml.contrib.automl.pipeline.steps.utilities import get_output_from_mm_pipeline

forecasting_results_name = "forecasting_results"
forecasting_output_name = "many_models_inference_output"
forecast_file = get_output_from_mm_pipeline(
    inference_run, forecasting_results_name, forecasting_output_name, output_file_name
)
df = pd.read_csv(forecast_file)
print(
    "Prediction has ", df.shape[0], " rows. Here the first 10 rows are being displayed."
)
df.head(10)

## 7.0 Publish and schedule the inference pipeline (Optional)

### 7.1 Publish the pipeline

Once you have a pipeline you're happy with, you can publish a pipeline so you can call it programmatically later on. See this [tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-your-first-pipeline#publish-a-pipeline) for additional information on publishing and calling pipelines.

In [ ]:
# published_pipeline_inf = inference_pipeline.publish(name = 'automl_forecast_many_models',
#                                                     description = 'forecast many models',
#                                                     version = '1',
#                                                     continue_on_step_failure = False)

### 7.2 Schedule the pipeline
You can also [schedule the pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-schedule-pipelines) to run on a time-based or change-based schedule. This could be used to automatically retrain or forecast models every month or based on another trigger such as data drift.

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence

# forecasting_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="automl_forecasting_recurring_schedule",
#                             description="Schedule Forecasting Pipeline to run on the first day of every week",
#                             pipeline_id=forecasting_pipeline_id,
#                             experiment_name=experiment.name,
#                             recurrence=recurrence)